# Double Debiased Machine Learning for Difference in Differences

Prof. Daniel de Abreu Pereira Uhr

## Conteúdo

* Double Debiased Machine Learning for Difference in Differences - Chang (2020)
  * Artigo seminal que aplica o framework DML (Double Machine Learning) ao desenho clássico de DiD com tratamento binário em um cenário 2x2.
  * https://python.plainenglish.io/double-debiased-ml-for-did-1-fd08bebcf033

* Difference-in-Differences with Continuous Treatment - Zhang, L. (2025)
  * Extensão do DiD para tratamento contínuo (dose/quantidade) em 2x2, ainda sob o framework DML.
  * DiD 2x2 contínuo

* Dynamic DML
  * O DynamicDML foi feito para tratamento sequencial geral; em staggered adoption (uma vez tratado, sempre tratado) ele pode funcionar, mas há armadilhas de identificação e de positividade.
  * O alvo padrão é um efeito contemporâneo $𝜃_0(X_0)$ do tratamento em $t$ sobre $Y_t$ condicionado ao histórico (via $W_t$, $T_{t−1}$, etc.).
  * Quer um efeito contemporâneo médio condicional ao histórico (policy-style CATE/ATE) e há variação de switchers? → DynamicDML pode ser adequado (idealmente com $ΔT_t$).




## Referências

https://python.plainenglish.io/double-debiased-ml-for-did-1-fd08bebcf033


**Principais:**
* Chang, Neng-Chieh. Double/debiased machine learning for difference-in-differences models. The Econometrics Journal, Volume 23, Issue 2, May 2020, Pages 177–191, https://doi.org/10.1093/ectj/utaa001
* Zhang, L. (2025). Continuous difference-in-differences with double/debiased machine learning. https://arxiv.org/pdf/2408.10509
* https://docs.doubleml.org/stable/examples/py_double_ml_did.html
* Neng-Chieh Chang. (2023). Double Debiased Machine Learning for Difference in Differences. [arXiv:2301.11395v2](https://doi.org/10.48550/arXiv.2301.11395)
* Colangelo and Lee (2023). Double Debiased Machine Learning Nonparametric Inference with Continuous Treatments. [arXiv:2004.03036v8 ](https://doi.org/10.48550/arXiv.2004.03036) 
* GitHub: https://github.com/KColangelo/Double-ML-Continuous-Treatment


**Complementares:**
* Microsoft EconML: https://econml.azurewebsites.net/
* UBER CausalML: https://causalml.readthedocs.io/en/latest/
* https://docs.doubleml.org/stable/index.html
* https://github.com/MasaAsami/ReproducingDMLDiD/blob/main/notebook/Reproduction_of_DMLDiD_RO_for_NEW_SIMDATA.ipynb




**Observações:** O material apresentado aqui é uma adaptação do material de aula do Prof. Daniel de Abreu Pereira Uhr, e não deve ser utilizado para fins comerciais. O material é disponibilizado para fins educacionais e de pesquisa, e não deve ser reproduzido sem a devida autorização do autor. Este material pode conter erros e imprecisões. O autor não se responsabiliza por quaisquer danos ou prejuízos decorrentes do uso deste material. O uso deste material é de responsabilidade exclusiva do usuário. Caso você encontre erros ou imprecisões neste material, por favor, entre em contato com o autor para que possam ser corrigidos. O autor agradece qualquer *feedback* ou sugestão de melhoria.

---

In [1]:
import numpy as np
import pandas as pd

# Fixando seed para reprodutibilidade
np.random.seed(123)

# Parâmetros da simulação
n_individuals = 500   # número de indivíduos
n_periods = 5         # número de períodos por indivíduo

# Criação de IDs e períodos
ids = np.repeat(np.arange(n_individuals), n_periods)
periods = np.tile(np.arange(n_periods), n_individuals)

# Características fixas dos indivíduos (heterogeneidade)
X0 = np.random.normal(0, 1, size=(n_individuals, 2))
X0 = pd.DataFrame(np.repeat(X0, n_periods, axis=0), columns=["x0_1", "x0_2"])

# Variáveis de estado que evoluem no tempo (W)
W = pd.DataFrame({
    "w1": np.random.normal(0, 1, size=n_individuals * n_periods),
    "w2": np.random.binomial(1, 0.5, size=n_individuals * n_periods)
})

# Tratamento dinâmico T_t: depende do tempo e das variáveis de estado
T = (0.5 * W["w1"] + 0.3 * W["w2"] + np.random.normal(0, 1, size=len(W))).round().clip(0, 1)

# Outcome Y_t: efeito heterogêneo + controles + ruído
theta_0 = 2 * X0["x0_1"] - X0["x0_2"]
Y = theta_0 * T + 0.5 * W["w1"] + 0.2 * W["w2"] + np.random.normal(0, 1, size=len(T))

# Montando o dataframe final
df_dyn = pd.DataFrame({
    "group": ids,
    "period": periods,
    "Y": Y,
    "T": T,
    "w1": W["w1"],
    "w2": W["w2"],
    "x0_1": X0["x0_1"],
    "x0_2": X0["x0_2"]
})

df_dyn.head(10)


,group,period,Y,T,w1,w2,x0_1,x0_2
0,0,0,0.732202,0.0,-0.748827,1,-1.085631,0.997345
1,0,1,-0.333707,0.0,0.567595,1,-1.085631,0.997345
2,0,2,-0.411410,0.0,0.718151,0,-1.085631,0.997345
3,0,3,-2.990265,1.0,-0.999381,1,-1.085631,0.997345
4,0,4,-0.834117,1.0,0.474898,1,-1.085631,0.997345
5,1,0,0.807207,0.0,-1.868500,0,0.282978,-1.506295
6,1,1,1.111659,1.0,-0.202659,1,0.282978,-1.506295
7,1,2,0.062406,0.0,-1.134248,0,0.282978,-1.506295
8,1,3,-0.225126,-0.0,-0.807699,0,0.282978,-1.506295
9,1,4,-0.193330,0.0,-1.276077,0,0.282978,-1.506295


In [4]:
from econml.panel.dml import DynamicDML
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier

dynamic_dml_est = DynamicDML(
    model_y=RandomForestRegressor(),
    model_t=RandomForestClassifier(),
    discrete_treatment=True,
    cv=3,  # Número de folds para cross-fitting
    random_state=123
)


In [5]:
dynamic_dml_est = DynamicDML(
    model_y=RandomForestRegressor(),
    model_t=RandomForestClassifier(),
    discrete_treatment=True,
    cv=3,
    random_state=123
)

dynamic_dml_est.fit(Y_dyn, T_dyn, X=X_dyn, W=W_dyn, groups=groups)


'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
'force_all_finite' wa

In [ ]:
ate_inf = dynamic_dml_est.ate_inference(X_dyn)
print(ate_inf)

               Uncertainty of Mean Point Estimate              
mean_point stderr_mean zstat pvalue ci_mean_lower ci_mean_upper
---------------------------------------------------------------
     0.111       0.262 0.425  0.671        -0.403         0.625
      Distribution of Point Estimate     
std_point pct_point_lower pct_point_upper
-----------------------------------------
    2.308          -4.466           4.414
     Total Variance of Point Estimate     
stderr_point ci_point_lower ci_point_upper
------------------------------------------
       2.323         -4.502          4.579
------------------------------------------


'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.


In [2]:
from econml.panel.dml import DynamicDML
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier

# Separando variáveis
Y_dyn = df_dyn["Y"].values
T_dyn = df_dyn["T"].values.reshape(-1, 1)
X_dyn = df_dyn[["x0_1", "x0_2"]].values
W_dyn = df_dyn[["w1", "w2"]].values
groups = df_dyn["group"].values

# Estimador
dynamic_dml_est = DynamicDML(
    model_y=RandomForestRegressor(),
    model_t=RandomForestClassifier(),
    model_final=RandomForestRegressor(),
    discrete_treatment=True,
    cv=3,
    n_splits=3,
    random_state=123
)

# Ajustando o modelo
dynamic_dml_est.fit(Y_dyn, T_dyn, X=X_dyn, W=W_dyn, groups=groups)

# Estimando o efeito causal médio para todos os indivíduos
treatment_effects = dynamic_dml_est.effect(X_dyn)

# Resumo das estimativas
mean_effect = np.mean(treatment_effects)
std_effect = np.std(treatment_effects)

mean_effect, std_effect


TypeError: DynamicDML.__init__() got an unexpected keyword argument 'model_final'

### Double Debiased Machine Learning for Difference in Differences - Chang (2020)

* average treatment effect on the treated (ATT) under the conditional parallel trend assumption.
* based on Chang (2020), Sant’Anna and Zhao (2020) and Zimmert et al. (2018).

Nesse exmeplo a variável de tratamento e a variável de tempo $t\in\{0,1\}$ serão binárias.
Seja $D_i\in\{0,1\}$ o status de tratamento da unidade $i$ no tempo $t=1$ (no tempo $t=0$ todas as unidades não são tratadas) e seja $Y_{it}$ o resultado de interesse da unidade $i$ no tempo $t$.
Usando a notação de resultado potencial, podemos escrever $Y_{it}(d)$ para o resultado potencial da unidade $i$ no tempo $t$ e status de tratamento $d$. Além disso, seja $X_i$ um vetor de covariáveis pré-tratamento.
Nessa configuração de diferença em diferenças, o efeito médio do tratamento sobre os tratados (ATTE) é definido como (Abadie, 2005):

$$\theta = \mathbb{E}[Y_{i1}(1)- Y_{i1}(0)|D_i=1]$$

é identificado quando dados em painel estão disponíveis ou sob suposições de estacionaridade para seções transversais repetidas. Além disso, as suposições básicas são

 - **Parallel Trends:** We have $\mathbb{E}[Y_{i1}(0) - Y_{i0}(0)|X_i, D_i=1] = \mathbb{E}[Y_{i1}(0) - Y_{i0}(0)|X_i, D_i=0]\quad a.s.$

- **Overlap:** For some $\epsilon > 0$, $P(D_i=1) > \epsilon$ and $P(D_i=1|X_i) \le 1-\epsilon$ a.s.




In [1]:
import pandas as pd

df = pd.read_stata("https://github.com/Daniel-Uhr/data/raw/main/bacon_example.dta")

### 1. Introdução

Exemplo de aplicação do DDML-DiD


In [4]:
import pandas as pd
from differences import ATTgt

In [7]:
data = pd.read_stata("https://github.com/Daniel-Uhr/data/raw/main/bacon_example.dta")

In [8]:
# Filtragem dos dados
# Vamos criar identificadores estaduais
data['id'] = data['stfips'].astype('category').cat.codes + 1

# Outcome (Suicide Mortality)
data['Y'] = data['asmrs']
# Treatment
data['D'] = data['post']
# Covariáveis - pcinc asmrh cases
data['X1'] = data['pcinc']
data['X2'] = data['asmrh']
data['X3'] = data['cases']

# pcinc + asmrh + cases
# Vamos criar a variável de grupo G
data['G']=data['_nfd']

In [9]:
# Criar uma variável de tempo até o ano do início do tratamento (Tempo em relação ao início do evento)
data['timeToTreat'] = data['year'] - data['_nfd']
data['timeToTreat'].describe()

count    1188.000000
mean        6.416667
std        10.162403
min       -21.000000
25%        -2.000000
50%         6.000000
75%        15.000000
max        27.000000
Name: timeToTreat, dtype: float64

In [10]:
# O pacote precisa entender a estrutura de painel dos dados. Precisamos ajustar a variável year para int (inteiro)
# Criando clones
data['year1'] = data['year']
data['id1'] = data['id']
data['year1'] = data['year'].astype(int)

# Definir os indices (estrutura de painel)
data.set_index(['id1', 'year1'], inplace=True)

In [11]:
att_gt = ATTgt(data=data, cohort_name="G")

In [12]:
att_gt.fit("Y ~ pcinc", est_method="dr")

Computing ATTgt [workers=1]   100%|████████████████████| 384/384 [00:01<00:00, 198.30it/s]


ATTgtResult                                            \
                                     analytic pointwise conf. band              
                                ATT std_error                lower      upper   
cohort base_period time                                                         
1969   1964        1965   -6.070418  8.259573           -22.258884  10.118048   
       1965        1966    6.174149  9.094903           -11.651533  23.999831   
       1966        1967    5.848175  3.752705            -1.506991  13.203342   
       1967        1968    2.036093  2.414729            -2.696688   6.768875   
       1968        1969    1.558169  4.865841            -7.978705  11.095043   
...                             ...       ...                  ...        ...   
1985   1984        1992   19.930538  4.051701            11.989349  27.871726   
                   1993   16.124177  2.333263            11.551066  20.697287   
                   1994   11.830665  2.834945             6.274274  17.387055   
                   1995    8.493860  2.493009             3.607652  13.380068   
                   1996   20.431504  4.480529            11.649828  29.213179   

                                           
                                           
                        zero_not_in_cband  
cohort base_period time                    
1969   1964        1965                    
       1965        1966                    
       1966        1967                    
       1967        1968                    
       1968        1969                    
...                                   ...  
1985   1984        1992                 *  
                   1993                 *  
                   1994                 *  
                   1995                 *  
                   1996                 *  

[384 rows x 5 columns]

In [13]:
att_gt.aggregate("simple")

SimpleAggregation                                                           
                     analytic pointwise conf. band                            
                ATT std_error                lower     upper zero_not_in_cband
0         -5.983104  3.130347           -12.118472  0.152264

Rotina para Produzir Essa Tabela
Loop em cada G (coorte tratado).

Filtrar G e never-treated/not-yet-treated.

Estimar DoubleMLDiD para aquele G ao longo dos tempos ≥ G.

Agregação para cada G.

In [14]:
def estimate_att_gt(df, ml_g, ml_m):
    results = []
    cohorts = df['_nfd'].dropna().unique()
    times = df['year'].unique()

    for g in cohorts:
        for t in times:
            if t < g:
                continue  # apenas períodos pós-tratamento

            df_t = df[(df['year'] == t) & ((df['_nfd'] == g) | (df['_nfd'] > t) | (df['_nfd'].isna()))].copy()
            
            if df_t.empty or df_t['post'].sum() == 0 or (1 - df_t['post']).sum() == 0:
                continue

            X = df_t[['pcinc']].values
            Y = df_t['asmrs'].values
            D = df_t['post'].values

            dml_data = DoubleMLData.from_arrays(x=X, y=Y, d=D)
            dml_did = DoubleMLDID(dml_data,
                                  ml_g=ml_g,
                                  ml_m=ml_m,
                                  score='observational',
                                  in_sample_normalization=True,
                                  n_folds=5)
            dml_did.fit()
            summary = dml_did.summary

            att = summary['coef'].values[0]
            stderr = summary['std err'].values[0]
            ci_lower = summary['2.5 %'].values[0]
            ci_upper = summary['97.5 %'].values[0]

            # Conta quantos tratados existem nesse (G,t)
            n_treated = df_t['post'].sum()

            results.append({'G': g, 't': t, 'ATT': att, 'StdErr': stderr,
                            'CI Lower': ci_lower, 'CI Upper': ci_upper,
                            'n_treated': n_treated})

    return pd.DataFrame(results)


In [22]:
def aggregate_att_gt_weighted(results_df):
    weights = results_df['n_treated']
    weighted_att = np.average(results_df['ATT'], weights=weights)
    weighted_var = np.average(results_df['StdErr']**2, weights=weights)
    weighted_se = np.sqrt(weighted_var / len(results_df))

    ci_lower = weighted_att - 1.96 * weighted_se
    ci_upper = weighted_att + 1.96 * weighted_se

    summary = pd.DataFrame({
        'ATT Weighted Mean': [weighted_att],
        'StdErr Mean': [weighted_se],
        'CI Lower': [ci_lower],
        'CI Upper': [ci_upper]
    })

    return summary



In [23]:
from sklearn.linear_model import LinearRegression, LogisticRegression
from doubleml import DoubleMLData, DoubleMLDID
import numpy as np

ml_g_linear = LinearRegression()
ml_m_logit = LogisticRegression(max_iter=200)

att_gt_results = estimate_att_gt(data, ml_g=ml_g_linear, ml_m=ml_m_logit)
agg_summary = aggregate_att_gt_weighted(att_gt_results)

print(att_gt_results)
print(agg_summary)


          G       t        ATT    StdErr   CI Lower   CI Upper  n_treated
0    1971.0  1971.0  -4.221526  5.842524 -15.672663   7.229610       15.0
1    1971.0  1972.0  -6.282070  5.451026 -16.965885   4.401744       15.0
2    1971.0  1973.0  10.896126  6.422436  -1.691617  23.483870       15.0
3    1971.0  1974.0   9.245664  6.057875  -2.627552  21.118881       15.0
4    1971.0  1975.0  14.538667  6.652929   1.499167  27.578168       15.0
..      ...     ...        ...       ...        ...        ...        ...
253  1985.0  1992.0  -6.892958  9.499466 -25.511570  11.725654        9.0
254  1985.0  1993.0  10.149335  8.131817  -5.788733  26.087403        9.0
255  1985.0  1994.0   5.064727  6.063285  -6.819093  16.948547        9.0
256  1985.0  1995.0  13.008206  7.559949  -1.809021  27.825434        9.0
257  1985.0  1996.0   4.121333  3.797415  -3.321463  11.564129        9.0

[258 rows x 7 columns]
   ATT Weighted Mean  StdErr Mean  CI Lower  CI Upper
0           8.231945     0.555556 

In [121]:
ml_g_linear = LinearRegression()
ml_m_logit = LogisticRegression(max_iter=200)

att_gt_results = estimate_att_gt(df, ml_g=ml_g_linear, ml_m=ml_m_logit)
print(att_gt_results)

          G       t        ATT     StdErr   CI Lower   CI Upper
0    1971.0  1971.0  -5.020802   6.135667 -17.046488   7.004884
1    1971.0  1972.0  -9.813400   5.437082 -20.469886   0.843086
2    1971.0  1973.0  10.534043   6.398119  -2.006040  23.074126
3    1971.0  1974.0  11.216579   6.533464  -1.588774  24.021932
4    1971.0  1975.0  12.134260   6.754665  -1.104641  25.373161
..      ...     ...        ...        ...        ...        ...
253  1985.0  1992.0   0.796086  10.042278 -18.886417  20.478589
254  1985.0  1993.0   8.908715   7.161766  -5.128089  22.945518
255  1985.0  1994.0   5.705179   7.641421  -9.271730  20.682089
256  1985.0  1995.0  11.118021   7.309105  -3.207562  25.443605
257  1985.0  1996.0   7.528995   4.611464  -1.509308  16.567298

[258 rows x 6 columns]


In [122]:
def aggregate_att_gt(results_df):
    """
    Aggregate ATTgt estimates by averaging over all group-time pairs,
    and compute the empirical standard error of the mean.
    """
    # Média dos ATTs
    att_mean = results_df['ATT'].mean()

    # Desvio padrão empírico dos ATTs
    sd_att = results_df['ATT'].std(ddof=1)

    # Número de estimativas
    n_estimates = results_df.shape[0]

    # Erro padrão da média
    se_mean = sd_att / np.sqrt(n_estimates)

    # Intervalo de confiança 95%
    ci_lower = att_mean - 1.96 * se_mean
    ci_upper = att_mean + 1.96 * se_mean

    # Monta o resultado em DataFrame
    summary = pd.DataFrame({
        'ATT Mean': [att_mean],
        'StdErr Mean': [se_mean],
        'CI Lower': [ci_lower],
        'CI Upper': [ci_upper]
    })

    return summary


In [123]:
agg_summary = aggregate_att_gt(att_gt_results)
print(agg_summary)


   ATT Mean  StdErr Mean  CI Lower  CI Upper
0  7.981915     0.385102  7.227115  8.736715


In [124]:
def aggregate_att_gt_weighted(results_df, df_original):
    """
    Aggregate ATTgt estimates weighted by the number of treated units in each (g, t).
    """
    weighted_ests = []
    weights = []

    for idx, row in results_df.iterrows():
        g = row['G']
        t = row['t']
        
        # Conta o número de unidades tratadas em g no tempo t
        n_treated = df_original[(df_original['_nfd'] == g) & (df_original['year'] == t)].shape[0]
        
        if n_treated == 0:
            continue  # pula se não houver tratados no tempo t para o grupo g

        weighted_ests.append(row['ATT'] * n_treated)
        weights.append(n_treated)

    # Calcula a média ponderada
    weighted_att_mean = np.sum(weighted_ests) / np.sum(weights)

    # Calcula o erro padrão empírico ponderado
    att_values = results_df['ATT']
    sd_att = att_values.std(ddof=1)
    n_estimates = len(att_values)
    se_mean = sd_att / np.sqrt(n_estimates)

    # Intervalo de confiança 95%
    ci_lower = weighted_att_mean - 1.96 * se_mean
    ci_upper = weighted_att_mean + 1.96 * se_mean

    summary = pd.DataFrame({
        'ATT Weighted Mean': [weighted_att_mean],
        'StdErr Mean': [se_mean],
        'CI Lower': [ci_lower],
        'CI Upper': [ci_upper]
    })

    return summary


In [125]:
agg_weighted_summary = aggregate_att_gt_weighted(att_gt_results, df)
print(agg_weighted_summary)


   ATT Weighted Mean  StdErr Mean  CI Lower   CI Upper
0           9.823624     0.385102  9.068824  10.578424


In [112]:
def summarize_group_results_empirical(group_results):
    # Média dos ATTs
    att_mean = group_results['ATT'].mean()

    # Desvio padrão das estimativas (empírico)
    sd_att = group_results['ATT'].std(ddof=1)
    n_groups = group_results.shape[0]
    se_mean = sd_att / np.sqrt(n_groups)

    # Intervalo de confiança 95%
    ci_lower = att_mean - 1.96 * se_mean
    ci_upper = att_mean + 1.96 * se_mean

    # Monta o resultado
    summary = pd.DataFrame({
        'ATT Mean': [att_mean],
        'StdErr Mean': [se_mean],
        'CI Lower': [ci_lower],
        'CI Upper': [ci_upper]
    })

    return summary



In [113]:
summary_empirical = summarize_group_results_empirical(group_results)
print(summary_empirical)



   ATT Mean  StdErr Mean  CI Lower  CI Upper
0  4.035823     0.272238  3.502237  4.569409


In [75]:
# criar nova base df1 quando: G==1973 ou "missing"
df1 = data[(data['G'] == 1973) | (data['G'].isna())]

In [83]:
# Reformatar X, Y, D conforme o DoubleML espera
X = df1[['pcinc', 'asmrh', 'cases']].values
Y = df1['Y'].values
D = df1['D'].values

# Preparar o DoubleMLData
dml_data = DoubleMLData.from_arrays(x=X, y=Y, d=D)

# Configurar modelos de ML com RandomForest
ml_g = RandomForestRegressor(n_estimators=100, random_state=123)
ml_m = RandomForestClassifier(n_estimators=100, random_state=123)

# Instanciar e ajustar o DoubleMLDID
dml_did = DoubleMLDID(dml_data,
                      ml_g=ml_g,
                      ml_m=ml_m,
                      score='observational',
                      in_sample_normalization=True,
                      n_folds=3)

# Rodar a estimação
dml_did.fit()

# Mostrar o resultado
print(dml_did.summary)

      coef   std err         t     P>|t|     2.5 %    97.5 %
d  0.11987  2.897706  0.041367  0.967003 -5.559528  5.799269


c:\Users\danie\AppData\Local\Programs\Python\Python311\Lib\site-packages\doubleml\utils\_checks.py:204: UserWarning: Propensity predictions from learner RandomForestClassifier(random_state=123) for ml_m are close to zero or one (eps=1e-12).
  warnings.warn(f'Propensity predictions from learner {str(learner)} for'


In [77]:
att_gt = ATTgt(data=df1, cohort_name="G")

In [78]:
att_gt.fit("Y ~ 1 + pcinc + asmrh + cases", est_method="dr")

Computing ATTgt [workers=1]     0%|                    | 0/32 [00:00<?, ?it/s]

Computing ATTgt [workers=1]   100%|████████████████████| 32/32 [00:00<00:00, 71.60it/s]


ATTgtResult                                  \
                                      analytic pointwise conf. band   
                                ATT  std_error                lower   
cohort base_period time                                               
1973   1964        1965  -13.387811   7.096693           -27.297073   
       1965        1966   -3.608137   8.672492           -20.605909   
       1966        1967    9.726261   7.509689            -4.992460   
       1967        1968   -6.589926   7.325320           -20.947290   
       1968        1969    6.707202   4.838605            -2.776290   
       1969        1970    1.439206   5.826626            -9.980770   
       1970        1971    0.059841   7.768446           -15.166033   
       1971        1972    1.237457   7.684925           -13.824719   
       1972        1973    0.063519   7.657225           -14.944366   
                   1974    3.481384   7.333599           -10.892207   
                   1975    0.365940  11.946922           -23.049597   
                   1976    0.298584   7.212508           -13.837673   
                   1977   -0.533187   5.966501           -12.227314   
                   1978   -4.867036   6.188719           -16.996703   
                   1979   -5.218578   6.296130           -17.558767   
                   1980   -9.889103   8.384045           -26.321529   
                   1981   -3.990705   6.480583           -16.692414   
                   1982    3.563092   9.048286           -14.171224   
                   1983    1.610599   5.293055            -8.763598   
                   1984   -2.117688   7.497366           -16.812256   
                   1985   -0.798061   5.225490           -11.039833   
                   1986   -1.583932   5.458440           -12.282278   
                   1987   -1.914863   9.529503           -20.592345   
                   1988   -2.064580   8.745797           -19.206026   
                   1989   -4.327468   5.188970           -14.497662   
                   1990   -6.890669   5.637114           -17.939209   
                   1991   -6.460169   7.222601           -20.616207   
                   1992   -9.392253   5.919874           -20.994992   
                   1993   -6.705061   6.689964           -19.817149   
                   1994   -3.725380   6.106506           -15.693913   
                   1995   -9.408144   5.305199           -19.806143   
                   1996  -10.933998   5.361001           -21.441366   

                                                      
                                                      
                             upper zero_not_in_cband  
cohort base_period time                               
1973   1964        1965   0.521452                    
       1965        1966  13.389635                    
       1966        1967  24.444982                    
       1967        1968   7.767438                    
       1968        1969  16.190695                    
       1969        1970  12.859183                    
       1970        1971  15.285716                    
       1971        1972  16.299633                    
       1972        1973  15.071405                    
                   1974  17.854975                    
                   1975  23.781478                    
                   1976  14.434840                    
                   1977  11.160940                    
                   1978   7.262630                    
                   1979   7.121611                    
                   1980   6.543323                    
                   1981   8.711003                    
                   1982  21.297407                    
                   1983  11.984795                    
                   1984  12.576879                    
                   1985   9.443711                    
                   1986   9.114414                    
                   1987  16.762620   

In [79]:
att_gt.aggregate("simple")

SimpleAggregation                                                           
                     analytic pointwise conf. band                            
                ATT std_error                lower     upper zero_not_in_cband
0          -3.39324  5.373676           -13.925451  7.138971

In [89]:
from sklearn.linear_model import LinearRegression, LogisticRegression
from doubleml import DoubleMLDID, DoubleMLData

# Reformatar X, Y, D conforme o DoubleML espera
X = df1[['pcinc', 'asmrh', 'cases']].values
Y = df1['Y'].values
D = df1['D'].values

# Preparar o DoubleMLData
dml_data = DoubleMLData.from_arrays(x=X, y=Y, d=D)

# Configurar modelos de ML com RandomForest
ml_g_linear = LinearRegression()
ml_m_logit = LogisticRegression(max_iter=1000)

# Instanciar e ajustar o DoubleMLDID
dml_did = DoubleMLDID(dml_data,
                      ml_g=ml_g,
                      ml_m=ml_m,
                      score='observational',
                      in_sample_normalization=True,
                      n_folds=7)

# Rodar a estimação
dml_did.fit()

# Mostrar o resultado
print(dml_did.summary)

       coef   std err        t     P>|t|     2.5 %     97.5 %
d  4.976907  4.938242  1.00783  0.313536 -4.701869  14.655683


c:\Users\danie\AppData\Local\Programs\Python\Python311\Lib\site-packages\doubleml\utils\_checks.py:204: UserWarning: Propensity predictions from learner RandomForestClassifier(random_state=123) for ml_m are close to zero or one (eps=1e-12).
  warnings.warn(f'Propensity predictions from learner {str(learner)} for'


In [ ]:
from lightgbm import LGBMClassifier, LGBMRegressor

n_estimators = 30
ml_g = LGBMRegressor(n_estimators=n_estimators)
ml_m = LGBMClassifier(n_estimators=n_estimators)

In [ ]:
from doubleml import DoubleMLDID
dml_did = DoubleMLDID(dml_data,
                      ml_g=ml_g,
                      ml_m=ml_m,
                      score='observational',
                      in_sample_normalization=True,
                      n_folds=5)

dml_did.fit()
print(dml_did)

Rodar o DML na nova base df1

In [15]:
import numpy as np

class DoubleMLDiDStaggered:
    def __init__(self, data, yname, tname, gname, idname, xnames, ml_g, ml_m):
        """
        Initializes the DoubleMLDiDStaggered estimator.

        Parameters:
        - data: pandas DataFrame containing the panel data.
        - yname: str, outcome variable name.
        - tname: str, time variable name.
        - gname: str, treatment cohort variable name (first treated period).
        - idname: str, unit identifier variable name.
        - xnames: list of str, covariate names.
        - ml_g: fitted machine learning model for outcome regression.
        - ml_m: fitted machine learning model for propensity score estimation.
        """
        self.data = data.copy()
        self.yname = yname
        self.tname = tname
        self.gname = gname
        self.idname = idname
        self.xnames = xnames
        self.ml_g = ml_g
        self.ml_m = ml_m
        self.results = []

    def _fit_group_time(self, g, t):
        """
        Estimate ATT for a specific group-time pair.
        """
        df = self.data.copy()
        df['G'] = (df[self.gname] == g).astype(int)
        df['D'] = (df[self.tname] >= g).astype(int) * df['G']
        df['Post'] = (df[self.tname] == t).astype(int)
        df['Treat'] = df['D'] * df['Post']

        # Select treated and control units at time t
        df_t = df[df[self.tname] == t].copy()
        treated = df_t['G'] == 1
        control = (df_t[self.gname] > t) | (df_t[self.gname].isna())

        # Skip if no control or treated units
        if treated.sum() == 0 or control.sum() == 0:
            return None

        X = df_t[self.xnames]
        Y = df_t[self.yname]

        # Fit nuisance models on control units only (orthogonalization)
        self.ml_g.fit(X[control], Y[control])
        self.ml_m.fit(X, df_t['G'])

        mu0 = self.ml_g.predict(X)
        pscore = np.clip(self.ml_m.predict_proba(X)[:, 1], 1e-6, 1 - 1e-6)

        # Compute DR scores
        dr_scores = ((df_t['G'] - pscore) / pscore / (1 - pscore)) * (Y - mu0)

        att = dr_scores[treated].mean()
        return {'g': g, 't': t, 'att': att}

    def fit(self):
        """
        Fit the model across all group-time combinations.
        """
        groups = self.data[self.gname].dropna().unique()
        times = self.data[self.tname].unique()

        results = []
        for g in groups:
            for t in times:
                if t >= g:
                    res = self._fit_group_time(g, t)
                    if res is not None:
                        results.append(res)
        self.results = pd.DataFrame(results)
        return self

    def aggregate_att(self):
        """
        Aggregate the ATT estimates by averaging over all group-time pairs.
        """
        return self.results['att'].mean()

    def plot_event_study(self):
        """
        Plot the event-study style results.
        """
        import matplotlib.pyplot as plt

        if self.results.empty:
            print("No results to plot.")
            return

        avg_att_by_time = self.results.groupby('t')['att'].mean()
        plt.figure(figsize=(8, 5))
        plt.plot(avg_att_by_time.index, avg_att_by_time.values, marker='o')
        plt.axhline(0, color='black', linestyle='--')
        plt.xlabel('Time')
        plt.ylabel('Average ATT')
        plt.title('Event Study: ATT over Time')
        plt.show()


# Pronto para usar. Você quer que eu monte um exemplo usando o arquivo `bacon_example.dta`?


In [17]:
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier

# Configuração dos modelos de ML
ml_g = RandomForestRegressor(n_estimators=100, random_state=123)
ml_m = RandomForestClassifier(n_estimators=100, random_state=123)

# Configuração do estimador com os dados carregados
dml_did_staggered = DoubleMLDiDStaggered(
    data=data,
    yname="asmrs",
    tname="year",
    gname="_nfd",
    idname="stfips",
    xnames=["pcinc", "asmrh", "cases"],
    ml_g=ml_g,
    ml_m=ml_m
)

# Rodar o estimador completo
dml_did_staggered.fit()

# Exibir resultados
dml_did_staggered.results


,g,t,att
0,1971.0,1971.0,-6.428095
1,1971.0,1972.0,-15.785884
2,1971.0,1973.0,-5.387281
3,1971.0,1974.0,16.245868
4,1971.0,1975.0,2.767154
...,...,...,...
253,1985.0,1992.0,11.478867
254,1985.0,1993.0,-6.032115
255,1985.0,1994.0,-14.044631
256,1985.0,1995.0,-23.927145


In [20]:
from sklearn.linear_model import LinearRegression, LogisticRegression

# Configuração dos modelos paramétricos simples (Linear e Logit)
ml_g_linear = LinearRegression()
ml_m_logit = LogisticRegression(max_iter=1000)

# Novo estimador usando modelos simples
dml_did_staggered_simple = DoubleMLDiDStaggered(
    data=data,
    yname="asmrs",
    tname="year",
    gname="_nfd",
    idname="stfips",
    xnames=["pcinc", "asmrh", "cases"],
    ml_g=ml_g_linear,
    ml_m=ml_m_logit
)

# Rodar o estimador completo (agora leve)
dml_did_staggered_simple.fit()

dml_did_staggered_simple.results

,g,t,att
0,1971.0,1971.0,-31.346567
1,1971.0,1972.0,-54.913850
2,1971.0,1973.0,-21.940507
3,1971.0,1974.0,63.389741
4,1971.0,1975.0,11.062567
...,...,...,...
253,1985.0,1992.0,57.538368
254,1985.0,1993.0,-101.236223
255,1985.0,1994.0,-309.071009
256,1985.0,1995.0,-139.452963


In [24]:
import numpy as np
from doubleml.datasets import make_did_SZ2020
from doubleml import DoubleMLData

np.random.seed(42)
n_obs = 1000
x, y, d = make_did_SZ2020(n_obs=n_obs, dgp_type=4, cross_sectional_data=False, return_type='array')
dml_data = DoubleMLData.from_arrays(x=x, y=y, d=d)
print(dml_data)

================== DoubleMLData Object ==================

------------------ Data summary      ------------------
Outcome variable: y
Treatment variable(s): ['d']
Covariates: ['X1', 'X2', 'X3', 'X4']
Instrument variable(s): None
No. Observations: 1000

------------------ DataFrame info    ------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Columns: 6 entries, X1 to d
dtypes: float64(6)
memory usage: 47.0 KB



In [26]:
from lightgbm import LGBMClassifier, LGBMRegressor

n_estimators = 30
ml_g = LGBMRegressor(n_estimators=n_estimators)
ml_m = LGBMClassifier(n_estimators=n_estimators)

In [27]:
from doubleml import DoubleMLDID
dml_did = DoubleMLDID(dml_data,
                      ml_g=ml_g,
                      ml_m=ml_m,
                      score='observational',
                      in_sample_normalization=True,
                      n_folds=5)

dml_did.fit()
print(dml_did)

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0,000450 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 532
[LightGBM] [Info] Number of data points in the train set: 396, number of used features: 4
[LightGBM] [Info] Start training from score 218,891977
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -

In [ ]:
# Recriar o dataset simulado conforme seu exemplo anterior
import numpy as np
import pandas as pd
from doubleml.datasets import make_did_SZ2020
from differences import ATTgt

# Simular os dados
np.random.seed(42)
n_obs = 1000
x, y, d = make_did_SZ2020(n_obs=n_obs, dgp_type=4, cross_sectional_data=False, return_type='array')

# Criar estrutura de painel artificial
df = pd.DataFrame(x, columns=[f'x{i+1}' for i in range(x.shape[1])])
df['Y'] = y
df['D'] = d

# Adicionar período artificial: 0 (pré) e 1 (pós), duplicando os dados
df_pre = df.copy()
df_pre['time'] = 0
df_pre['D'] = 0  # todos ainda não tratados no pré

df_post = df.copy()
df_post['time'] = 1
df_post['D'] = d  # tratamento no pós conforme definido

# Empilhar o painel
panel_df = pd.concat([df_pre, df_post], ignore_index=True)

# Adicionar ids
panel_df['id'] = np.tile(np.arange(1, n_obs + 1), 2)

# Definir cohort: todos tratados no tempo 1 se D == 1, NaN caso contrário
panel_df['G'] = panel_df.apply(lambda row: 1 if row['D'] == 1 and row['time'] == 1 else np.nan, axis=1)

# Definir índice de painel
panel_df = panel_df.set_index(['id', 'time'])

# Rodar o ATTgt no pacote differences
att_gt = ATTgt(data=panel_df, cohort_name="G")

# Como é apenas um "cohort", usamos uma fórmula simples sem covariáveis adicionais
att_gt.fit("Y ~ 1", est_method="reg")

Computing ATTgt [workers=1]   100%|████████████████████| 1/1 [00:00<00:00, 61.69it/s]


SimpleAggregation                                                       
                     analytic pointwise conf. band                        
                ATT std_error                lower upper zero_not_in_cband
0               0.0       NaN                  NaN   NaN

In [ ]:
# Agregar o ATT médio
att_gt

In [32]:
panel_df


,,x1,x2,x3,x4,Y,D,G
id,time,,,,,,,
1,0,0.248273,-0.117178,0.309346,0.921686,251.864668,0.0,NaN
2,0,-0.435201,-0.264080,-0.392549,0.301735,232.226706,0.0,NaN
3,0,-0.607181,0.606272,0.199458,-0.020248,190.547905,0.0,NaN
4,0,-0.018766,-1.595768,-0.438489,-1.687260,159.658966,0.0,NaN
5,0,-0.934856,0.412742,0.969647,-0.815696,155.990454,0.0,NaN
...,...,...,...,...,...,...,...,...
996,1,1.457921,0.607361,5.718933,0.290200,298.238992,0.0,NaN
997,1,-0.017862,-1.733995,0.112738,-1.781041,188.432492,0.0,NaN
998,1,2.572294,-0.204188,-2.668426,-0.912382,219.081292,1.0,1.0


segundo exemplo de aplicação do DDML-DiD

# Double Debiased Machine Learning for Continuous Treatments

## Introdução

O **Double Debiased Machine Learning (DML)** é um método estatístico robusto e eficiente para estimar efeitos causais em contextos onde o tratamento é uma variável contínua. Este método combina:
- **Momentos Duplamente Robustos**, que garantem consistência mesmo quando uma das funções auxiliares é mal especificada.
- **Cross-fitting**, para evitar viés de overfitting causado pelo uso do mesmo conjunto de dados para treinamento e inferência.
- **Métodos de Machine Learning (ML)**, como LASSO, Redes Neurais ou Random Forests, para modelar funções de expectativa condicional e densidades condicionais.

O método é especialmente adequado para cenários onde o número de covariáveis é grande (alta dimensionalidade), sendo robusto a especificações incorretas de uma das funções auxiliares.

---

## Estrutura do Problema

### Dados e Objetivos

Considere uma amostra $\{Y_i, T_i, X_i\}_{i=1}^n$, onde:
- $Y_i$: Desfecho de interesse (variável dependente).
- $T_i$: Tratamento contínuo.
- $X_i$: Covariáveis observadas (potencialmente de alta dimensionalidade).

O objetivo principal é estimar:
1. **Função de Resposta Média à Dose (Average Dose-Response Function - ADRF)**:
   $$
   \beta_t = \mathbb{E}[Y(t)],
   $$
   onde $Y(t)$ representa o desfecho potencial para um valor específico $t$ do tratamento.

2. **Efeito Marginal (Partial Effect)**:
   $$
   \theta_t = \frac{\partial \beta_t}{\partial t}.
   $$

---

## Suposições para Identificação

Para identificar os efeitos causais, assumimos:
1. **Independência Condicional (Unconfoundedness)**:
   $$
   T \perp \varepsilon \mid X,
   $$
   onde $\varepsilon$ é o erro não observado. Esta suposição implica que, condicional nas covariáveis $X$, o tratamento é independentemente alocado.

2. **Suporte Comum (Common Support)**:
   $$
   f_{T|X}(t \mid X) > 0, \quad \forall t \in \mathcal{T}.
   $$
   Esta condição assegura que há sobreposição suficiente entre grupos de tratamento.

---

## Estimador DML

O estimador baseia-se em uma função momento duplamente robusta, definida como:
$$
\psi_t(Y_i, T_i, X_i) = \gamma(t, X_i) + \frac{K_h(T_i - t)}{f_{T|X}(t \mid X_i)} \left(Y_i - \gamma(t, X_i)\right),
$$
onde:
- $\gamma(t, X) = \mathbb{E}[Y \mid T = t, X]$: Expectativa condicional.
- $f_{T|X}(t \mid X)$: Densidade condicional do tratamento.
- $K_h(T_i - t)$: Função kernel para ponderar observações próximas do valor de tratamento $t$.

O estimador para $\beta_t$ é dado por:
$$
\hat{\beta}_t = \frac{1}{n} \sum_{i=1}^n \psi_t(Y_i, T_i, X_i).
$$

### Cross-Fitting
Para evitar viés de overfitting, os dados são particionados em $L$ subconjuntos (folds). Para cada fold $\ell$, as funções $\gamma$ e $f_{T|X}$ são estimadas usando apenas os dados fora do fold $\ell$. O estimador final é obtido pela média das estimativas em cada fold:
$$
\hat{\beta}_t = \frac{1}{n} \sum_{i=1}^n \left[ \gamma_{-\ell}(t, X_i) + \frac{K_h(T_i - t)}{f_{T|X,-\ell}(t \mid X_i)} \left(Y_i - \gamma_{-\ell}(t, X_i)\right) \right].
$$

---

## Propriedades Assintóticas

Sob condições regulares, o estimador $\hat{\beta}_t$ é:
1. **Consistente**: Converge para o valor verdadeiro $\beta_t$.
2. **Assintoticamente Normal**:
   $$
   \sqrt{n} (\hat{\beta}_t - \beta_t) \xrightarrow{d} N(0, V_t),
   $$
   onde $V_t$ é a variância assintótica, que pode ser estimada como:
   $$
   \hat{V}_t = \frac{1}{n^2} \sum_{i=1}^n \psi_t^2(Y_i, T_i, X_i).
   $$

---

## Estimação do Efeito Marginal

O efeito marginal $\theta_t$ é estimado numericamente:
$$
\hat{\theta}_t = \frac{\hat{\beta}_{t + \eta/2} - \hat{\beta}_{t - \eta/2}}{\eta},
$$
onde $\eta$ é uma sequência positiva que converge para zero à medida que $n \to \infty$.

Para garantir consistência e eficiência, $\eta$ deve ser escolhida adequadamente, levando em conta o tamanho amostral e a variabilidade nas estimativas de $\beta_t$.

---

## Benefícios e Limitações

### Benefícios
- **Flexibilidade**: Permite o uso de métodos de aprendizado de máquina para modelar $\gamma$ e $f_{T|X}$.
- **Eficiência**: Utiliza técnicas como cross-fitting para melhorar a precisão das estimativas.
- **Robustez**: Resistente a erros de especificação em uma das funções auxiliares.

### Limitações
- **Complexidade Computacional**: Requer a estimativa de funções auxiliares de alta dimensionalidade.
- **Sensibilidade ao Kernel**: A escolha do kernel e da largura de banda ($h$) pode impactar os resultados.

---

## Exemplo Intuitivo

Considere um estudo que analisa o impacto de **horas de treinamento em um programa de capacitação** ($T$) nos **salários futuros** ($Y$):
- Usamos $\gamma(t, X)$ para prever salários dados $T$ e as covariáveis $X$ (e.g., idade, escolaridade).
- Estimamos $f_{T|X}(t \mid X)$, que captura a distribuição das horas de treinamento, dado o perfil do indivíduo.
- O DML ajusta as estimativas para isolar o impacto causal de $T$ em $Y$, mesmo que algumas relações sejam complexas ou não lineares.

---

## Considerações Finais
O Double Debiased Machine Learning é um método avançado para estimar efeitos causais com tratamentos contínuos, oferecendo:

* Robustez a especificações incorretas de modelos auxiliares.
* Flexibilidade no uso de métodos de ML para modelagem de alta dimensionalidade.
* Garantias teóricas de consistência e normalidade assintótica.

Este método é ideal para aplicações empíricas que exigem alto rigor estatístico, como estudos em economia, saúde e políticas públicas.

---
